## Higgs Effective Field Theory (HEFT) Study: Prepare Training Data
> Created: Feb 15, 2024 Nicola de Filippis, Kurtis Johnson, Harrison B. Prosper<br>

### Introduction

In this project, the physics model of interest is the Higgs effective field theory (HEFT), defined by a 5D parameter space of Wilson coefficients [1]:
\begin{align}
    \theta & = c_{hhh}, c_t, c_{tt}, c_{ggh}, c_{gghh}.
\end{align}
For this investigation, we restrict our attention to a single observable, namely, the di-Higgs mass denoted by $m_{hh}$ and, in our proof-of-principle, we set $c_{hhh} = c_t = 1$. The Standard Model (SM) values for these parameters are $c_{hhh} = c_t = 1$, and $c_{tt} = c_{ggh} = c_{gghh} = 0$. In general, we would consider multiple observables simultaneously.

We shall use simulation-based inference to construct **confidence sets**, at confidence level (CL) $\tau$, in the HEFT and later SMEFT parameter spaces. This requires approximating $\mathbb{P}(\lambda \le \lambda_0 | \theta) = \mathbb{E}(Z | \theta)$ [2,3], which we shall do in two different ways: 1) using a multi-dimensional histogram constructed using a kdtree and 2) a machine learning (ML) model, where for a given hypothesis $H_0: \theta = \theta_0$ versus $H_1: \theta \ne \theta_0$, $\lambda_0$ is the observed value of a test statistic $\lambda$ with the property that large values of the test statistic *disfavor* the hypothesis $H_0$. 

Given the cumulative distribution function (cdf), $\mathbb{P}(\lambda \le \lambda_0 | \theta)$, a confidence set at CL $\tau$ is the set of $\theta$ values for which $\mathbb{P}(\lambda \le \lambda_0 | \theta) \leq \tau$.  By definition, a confidence set (in the simplest case in which we restrict ourselves to a single class of experiments) is an observation-dependent set of $\theta$ values which upon repeated replication of the experiment is guaranteed to include the true value of $\theta$ a fraction $\ge \tau$.


### Training data
In principle [2,3], to approximate $\mathbb{P}(\lambda \le \lambda_0 | \theta)$ we need to sample the HEFT parameter space sufficiently densely and, at every point, simulate data like the ones actually observed. However, if it is computationally infeasible to sample the parameter space densely enough, the next option is to use a sparser sampling of the parameter space and reweight existing simulated data to mimic the sampling of data at any other point $\theta$. To do this requires knowledge of the cross section per binned observables, or the differential cross section,  as a function of both the *observables* and the *parameters*; here, $m_{hh}$ and $\theta$, respectively. Therefore, one sub-goal of the project is construct a parametrization of this function.

This notebook prepares the data needed to train a model of the di-Higgs cross section as a function of $m_{hh}$ and the Wilson coefficients, $\theta$.

### References
  1. Lina Alasfar *et al.*, arXiv:2304.01968v1
  2. Ann Lee *et al.*, https://arxiv.org/abs/2107.03920
  3. Ali Al Kadhim *et al.*, https://iopscience.iop.org/article/10.1088/2632-2153/ad218e


In [1]:
import os, sys

# the standard module for tabular data
import pandas as pd

# the standard module for array manipulation
import numpy as np

# the standard modules for high-quality plots
import matplotlib as mp
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# update fonts
FONTSIZE = 16
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : FONTSIZE}
mp.rc('font', **font)

# set usetex = False if LaTex is not 
# available on your system or if the 
# rendering is too slow
mp.rc('text', usetex=True)

### Load $\texttt{POWHEG}$ data

Note 1: $\kappa_\lambda \equiv c_{hhh}$.

Note 2: The bin widths in $m_{hh}$ is 15 GeV.

In [3]:
def read_data(datafiles):
    df = []
    for datafile in datafiles:
        print('reading %s' % datafile)
        df.append( pd.read_csv(datafile) )

    # concatenate dataframes
    df = pd.concat(df)
    
    # select points with klambda=CT=1
    select = (df.klambda==1) * (df.CT==1)
    
    # make number of rows be a multiple of 20
    total  = select.sum()
    total  = int(total / 20)
    total  = 20 * total
    
    df = df[select][:total]
    print('\nnumber of rows read: %d\n' % len(df))

    # randomly shuffle order of rows in dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    return df

In [4]:
datafiles = ['../data/powheg_total_param_closeBP.csv', 
             '../data/powheg_total_param_closeBP_all.csv']

dfBP = read_data(datafiles)
dfBP[:5]

reading ../data/powheg_total_param_closeBP.csv
reading ../data/powheg_total_param_closeBP_all.csv

number of rows read: 340



,klambda,CT,CTT,CGHH,CGGHH,0,1,2,3,4,...,92,93,94,95,96,97,98,99,100,101
0,1.0,1.0,0.5,-0.6000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.000214,0.000166,0.000171,0.000160,0.000139,0.000092,0.000128,0.000123,0.000080,0.002084
1,1.0,1.0,0.5,0.1333,0.3333,0.0,0.0,0.0,0.0,0.0,...,0.000650,0.000890,0.000891,0.000729,0.000447,0.000649,0.000610,0.000323,0.000568,0.013908
2,1.0,1.0,-1.5,0.0000,0.3333,0.0,0.0,0.0,0.0,0.0,...,0.004181,0.002618,0.002612,0.001043,0.001569,0.002354,0.002877,0.002355,0.003135,0.046254
3,1.0,1.0,-1.0,0.8000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.000739,0.000592,0.001628,0.000590,0.000445,0.000888,0.000593,0.001034,0.000743,0.012307
4,1.0,1.0,0.5,-0.4000,0.3333,0.0,0.0,0.0,0.0,0.0,...,0.000509,0.000700,0.000730,0.000670,0.000509,0.000478,0.000668,0.000445,0.000670,0.014517


### Convert to $\texttt{numpy}$ arrays
 1. Get Wilson coefficients and cross sections / bin
 2. Protect against negative cross sections
 3. Pick cross sections from bin 17 (first non-zero bin) to 96

In [5]:
def get_column_names(df, first='17', last='96'):
    # get column names
    columns = list(df.columns)
    params  = columns[2:5]
    
    firstbin= columns.index(first)
    lastbin = columns.index(last)
    
    bins = columns[firstbin:lastbin+1]
    return params, bins

params, bins = get_column_names(dfBP)

params, bins[0], bins[-1], len(bins)

(['CTT', 'CGHH', 'CGGHH'], '17', '96', 80)

In [6]:
# get Wilson coefficients
wilson = dfBP[params].to_numpy()
print(wilson.shape)

# get cross section data
BP   = dfBP[bins].to_numpy()
print(BP.shape)

# protect against negative cross sections
BP   = np.where(BP < 0, 0, BP) 

print(f'cross section/bin (min, mean, max, std): '\
      f'{BP.min():8.4f} pb, {BP.mean():8.4f} pb, {BP.max():8.4f} pb, {BP.std():8.4f} pb')

BP[0]

(340, 3)
(340, 80)
cross section/bin (min, mean, max, std):   0.0000 pb,   0.1135 pb,   2.3421 pb,   0.2548 pb


array([6.94918563e-05, 4.08277789e-04, 5.24682924e-04, 4.43543890e-04,
       6.56655873e-04, 6.64794468e-04, 6.20978128e-04, 4.65427147e-04,
       9.12227901e-04, 1.10898679e-03, 1.78584363e-03, 2.24303175e-03,
       2.80485698e-03, 3.07088811e-03, 3.19527229e-03, 3.50517384e-03,
       3.77400499e-03, 4.18153778e-03, 3.84451752e-03, 4.14686557e-03,
       3.88311036e-03, 3.58228059e-03, 3.66736902e-03, 3.50612891e-03,
       3.51658394e-03, 3.23523418e-03, 3.18161771e-03, 2.92400364e-03,
       2.95633264e-03, 2.83325487e-03, 2.53711967e-03, 2.41407659e-03,
       2.27498123e-03, 2.09384738e-03, 1.94726430e-03, 1.87015824e-03,
       1.81162171e-03, 1.70694443e-03, 1.48662552e-03, 1.60637335e-03,
       1.53532205e-03, 1.26094534e-03, 1.30399968e-03, 1.22157205e-03,
       1.04781787e-03, 9.68385895e-04, 9.21520113e-04, 8.97049089e-04,
       9.99954063e-04, 8.43979185e-04, 6.78579032e-04, 7.16906972e-04,
       4.64403187e-04, 5.19794587e-04, 5.44592505e-04, 5.23999508e-04,
      

### Save shuffled spectra

In [7]:
# first place data in a dictionary
dmap = {'CTT': wilson.T[0], 
        'CGGH': wilson.T[1], 
        'CGGHH': wilson.T[2]}

dmap.update( {key: BP.T[i] for i, key in enumerate(bins)} )

# then save to a csv file
df = pd.DataFrame(dmap)
df.to_csv('../data/heft_spectra.csv', index=False)

len(df), df[:5]

(340,
    CTT    CGGH   CGGHH        17        18        19        20        21  \
 0  0.5 -0.6000  0.0000  0.000069  0.000408  0.000525  0.000444  0.000657   
 1  0.5  0.1333  0.3333  0.005106  0.030635  0.033326  0.032770  0.029841   
 2 -1.5  0.0000  0.3333  0.006264  0.050164  0.077592  0.109697  0.115487   
 3 -1.0  0.8000  0.0000  0.002070  0.019394  0.032846  0.054304  0.071467   
 4  0.5 -0.4000  0.3333  0.002099  0.012663  0.017212  0.018394  0.017857   
 
          22        23  ...        87        88        89        90        91  \
 0  0.000665  0.000621  ...  0.000230  0.000235  0.000171  0.000161  0.000171   
 1  0.030162  0.026020  ...  0.000933  0.001214  0.000730  0.000853  0.000686   
 2  0.142210  0.161502  ...  0.003657  0.003392  0.003662  0.002351  0.002873   
 3  0.090617  0.101578  ...  0.000931  0.001479  0.001774  0.001038  0.001777   
 4  0.016186  0.016581  ...  0.001463  0.000797  0.000732  0.001081  0.000667   
 
          92        93        94        95

### Save training data to a csv file

Format: $c_{tt}$, $c_{ggh}$, $c_{ggh}$, $m_{hh}$, $\sigma$

Save data from the last 300 spectra and keep the first 40 spectra to test the spectrum model.

In [11]:
# compute mid-points of di-Higgs mass bins, with the mass range mapped to the
# unit interval
_, xbins = BP.shape
xmin = 0
xmax = xbins/100 # bin width is 0.01, which corresponds to 15 GeV

x = np.linspace(xmin, xmax, xbins+1)
# x[1:] = x[1], x[2] ...,x[n-1]
# x[:-1]= x[0], x[1],...,x[n-2]
x = (x[1:]+x[:-1])/2

# fill list with training data 
START = 40
data  = []
for params, spectrum in zip(wilson[START:], BP[START:]):
    p = list(params)

    # increase the incidence of spectra with 
    # cross sections < 0.30 pb
    total_xsec = spectrum.sum()
    if total_xsec < 0.30:
        N = 6
    else:
        N = 1
        
    d = []
    for _ in range(N):
        for mhh, sigma in zip(x, spectrum):
            d = p + [mhh, sigma] # CTT, CGGH, CGGHH, MHH, SIGMA
            data.append(d)
print(f'training data size: {len(data):5d}')

# randomly shuffle rows
data = np.array(data)
np.random.shuffle(data)

# create dataframe
data = data.T  # transpose to shape (5, 25500)
df = pd.DataFrame({'CTT':   data[0], 
                   'CGGH':  data[1], 
                   'CGGHH': data[2], 
                   'mhh':   data[3], 
                   'sigma': data[4]})

# save training data to a csv file
df.to_csv('../data/heft_traindata.csv', index=False)

df

training data size: 42400


,CTT,CGGH,CGGHH,mhh,sigma
0,-3.0,0.6666,0.0666,0.105,1.178172
1,-3.0,-0.6000,-0.8000,0.625,0.026347
2,-1.0,0.8000,-0.8000,0.475,0.017784
3,-1.0,0.6666,0.0666,0.425,0.009975
4,-1.0,0.1333,0.0000,0.605,0.004213
...,...,...,...,...,...
42395,-1.0,0.1333,0.0000,0.185,0.104613
42396,0.0,0.2666,0.0000,0.055,0.002540
42397,0.5,0.2000,-0.2000,0.785,0.000398
42398,0.5,0.5333,-0.2000,0.785,0.000309
